# Doublet removal
Goal: ensure that each barcode corresponds to only one cell

In [ ]:
import scanpy as sc
import os
import scrublet as scr
from wand.image import Image as WImage

## Import parameters from snakemake

In [ ]:
sample = snakemake.wildcards.samples
scrub_threshold = snakemake.params.scrub_threshold

In [ ]:
sc.settings.autosave = True
sc.settings.set_figure_params(dpi=80)
figdir = f"4_Doublets/{sample}"
sc.settings.figdir = figdir
os.makedirs(figdir, exist_ok=True)

## 1. Load data

In [ ]:
adata = sc.read_h5ad(f"3_QC/{sample}_QC.h5ad")

## 2. Calculate doublet score for each cell

In [ ]:
scrub = scr.Scrublet(adata.raw.X)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets()

if scrub_threshold:
    adata.obs['predicted_doublets'] = scrub.call_doublets(threshold= scrub_threshold)
    print(f"Using manually set threshold: {scrub_threshold}")
histogram, axis = scrub.plot_histogram()


num_predicted_doublets = sum(adata.obs['predicted_doublets'])
print(f"Predicted {num_predicted_doublets} doublets")

histogram.savefig(f"{figdir}/histogram_{sample}_doublets.pdf")
img = WImage(filename=f"{figdir}/histogram_{sample}_doublets.pdf")
img

The histogram should show a bimodal distribution and the threshold shown in the "simulated doublets" plot should be at the minimum between the two modes. 

In [ ]:
# add in column with singlet/doublet instead of True/False
adata.obs['doublet_info'] = adata.obs["predicted_doublets"].astype(str)

## Plot number of detected genes in doublets vs singlets
It's expected that doublets/multiplets show more detected genes than a single cell  
True: doublets  
False: singlets

In [ ]:
sc.pl.violin(adata, 'n_genes_by_counts',
             jitter=0.4, groupby = 'doublet_info', rotation=45,
            save = f"_{sample}_ngenes_by_counts_doublets.pdf")

img = WImage(filename=f"{figdir}/violin_{sample}_ngenes_by_counts_doublets.pdf")
img

## Plot predicted doublets and doublet scores

In [ ]:
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
UMAP, axis = scrub.plot_embedding('UMAP', order_points=True);

UMAP.savefig(f"{figdir}/{sample}_doublets.pdf")

img = WImage(filename=f"{figdir}/{sample}_doublets.pdf")
img

In [ ]:
# also revert back to the raw counts as the main matrix in adata
adata = adata.raw.to_adata() 

adata = adata[adata.obs['doublet_info'] == 'False',:]
print(adata.shape)

## Remove predicted doublets from dataset and  save

In [ ]:
save_file = f'4_Doublets/{sample}_QC_doublets.h5ad'
adata.write_h5ad(save_file)